In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn import datasets
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA#主成分分析
from sklearn.mixture import GaussianMixture,BayesianGaussianMixture
from sklearn.naive_bayes import GaussianNB

from scipy.stats import multivariate_normal

from collections import Counter
import math

我们谈到了用 k-means 进行聚类的方法，这次我们来说一下另一个很流行的算法：Gaussian Mixture Model (GMM)。事实上，GMM 和 k-means 很像，不过 GMM 是学习出一些概率密度函数来（所以 GMM 除了用在 clustering 上之外，还经常被用于 density estimation ），简单地说，k-means 的结果是每个数据点被 assign 到其中某一个 cluster 了，而 GMM 则给出这些数据点被 assign 到每个 cluster 的概率，又称作 soft assignment 。

得出一个概率有很多好处，因为它的信息量比简单的一个结果要多，比如，我可以把这个概率转换为一个 score ，表示算法对自己得出的这个结果的把握。也许我可以对同一个任务，用多个方法得到结果，最后选取“把握”最大的那个结果；另一个很常见的方法是在诸如疾病诊断之类的场所，机器对于那些很容易分辨的情况（患病或者不患病的概率很高）可以自动区分，而对于那种很难分辨的情况，比如，49% 的概率患病，51% 的概率正常，如果仅仅简单地使用 50% 的阈值将患者诊断为“正常”的话，风险是非常大的，因此，在机器对自己的结果把握很小的情况下，会“拒绝发表评论”，而把这个任务留给有经验的医生去解决。

废话说了一堆，不过，在回到 GMM 之前，我们再稍微扯几句。我们知道，不管是机器还是人，学习的过程都可以看作是一种“归纳”的过程，在归纳的时候你需要有一些假设的前提条件，例如，当你被告知水里游的那个家伙是鱼之后，你使用“在同样的地方生活的是同一种东西”这类似的假设，归纳出“在水里游的都是鱼”这样一个结论。当然这个过程是完全“本能”的，如果不仔细去想，你也不会了解自己是怎样“认识鱼”的。另一个值得注意的地方是这样的假设并不总是完全正确的，甚至可以说总是会有这样那样的缺陷的，因此你有可能会把虾、龟、甚至是潜水员当做鱼。也许你觉得可以通过修改前提假设来解决这个问题，例如，基于“生活在同样的地方并且穿着同样衣服的是同一种东西”这个假设，你得出结论：在水里有并且身上长有鳞片的是鱼。可是这样还是有问题，因为有些没有长鳞片的鱼现在又被你排除在外了。

在这个问题上，机器学习面临着和人一样的问题，在机器学习中，一个学习算法也会有一个前提假设，这里被称作“归纳偏执 (bias)”（bias 这个英文词在机器学习和统计里还有其他许多的意思）。例如线性回归，目的是要找一个函数尽可能好地拟合给定的数据点，它的归纳偏执就是“满足要求的函数必须是线性函数”。一个没有归纳偏执的学习算法从某种意义上来说毫无用处，就像一个完全没有归纳能力的人一样，在第一次看到鱼的时候有人告诉他那是鱼，下次看到另一条鱼了，他并不知道那也是鱼，因为两条鱼总有一些地方不一样的，或者就算是同一条鱼，在河里不同的地方看到，或者只是看到的时间不一样，也会被他认为是不同的，因为他无法归纳，无法提取主要矛盾、忽略次要因素，只好要求所有的条件都完全一样──然而哲学家已经告诉过我们了：世界上不会有任何样东西是完全一样的，所以这个人即使是有无比强悍的记忆力，也绝学不到任何一点知识。

这个问题在机器学习中称作“过拟合 (Overfitting)”，例如前面的回归的问题，如果去掉“线性函数”这个归纳偏执，因为对于 N 个点，我们总是可以构造一个 N-1 次多项式函数，让它完美地穿过所有的这 N 个点，或者如果我用任何大于 N-1 次的多项式函数的话，我甚至可以构造出无穷多个满足条件的函数出来。如果假定特定领域里的问题所给定的数据个数总是有个上限的话，我可以取一个足够大的 N ，从而得到一个（或者无穷多个）“超级函数”，能够 fit 这个领域内所有的问题。然而这个（或者这无穷多个）“超级函数”有用吗？只要我们注意到学习的目的（通常）不是解释现有的事物，而是从中归纳出知识，并能应用到新的事物上，结果就显而易见了。

没有归纳偏执或者归纳偏执太宽泛会导致 Overfitting ，然而另一个极端──限制过大的归纳偏执也是有问题的：如果数据本身并不是线性的，强行用线性函数去做回归通常并不能得到好结果。难点正在于在这之间寻找一个平衡点。不过人在这里相对于（现在的）机器来说有一个很大的优势：人通常不会孤立地用某一个独立的系统和模型去处理问题，一个人每天都会从各个来源获取大量的信息，并且通过各种手段进行整合处理，归纳所得的所有知识最终得以统一地存储起来，并能有机地组合起来去解决特定的问题。这里的“有机”这个词很有意思，搞理论的人总能提出各种各样的模型，并且这些模型都有严格的理论基础保证能达到期望的目的，然而绝大多数模型都会有那么一些“参数”（例如 K-means 中的 k ），通常没有理论来说明参数取哪个值更好，而模型实际的效果却通常和参数是否取到最优值有很大的关系，我觉得，在这里“有机”不妨看作是所有模型的参数已经自动地取到了最优值。另外，虽然进展不大，但是人们也一直都期望在计算机领域也建立起一个统一的知识系统（例如语意网就是这样一个尝试）。

## 大错误theta = {}

此前theta在来回的调用之后没有初始化导致得到的值始终是一样的

* 在编写函数的时候，尽量不用全局变量，使函数功能性分离，只通过输入变量输入，返回输出值。
* 在编写类时，慎用全局变量。
* 在编写循环语句或重复调用的函数时，注意对相应的变量进行初始化

### set(a)函数
set()对原列表a去重，通过sort(key = a.index)，将a中的元素按照 a 中元素出现的顺序排序


In [8]:
print(np.linalg.det(np.eye(3)))#单位矩阵的模

a1 = [3,5,2,5,6,1]
print(list(set(a1)))#生成去重排序后的列表

1.0
[1, 2, 3, 5, 6]


In [20]:
class GMM():
    def __init__(self,n_components = 5, max_iter = 10):
        self.k = n_components
        self.max_iter = max_iter
        
#     def GMM_component(self, X, theta, param, c):
#         for i in range(param['dim']):
#             for j in range(param['dim']):
#                 if np.isnan(theta['sigma'][c,i,j]):
#                     theta['sigma'][c,i,j] = 0
#         theta['sigma'][c, ...] += self.regularization
#         return theta['pi'][c]*multivariate_normal(theta['mu'][c], theta['sigma'][c, ...], allow_singular = True).pdf(X)
 
#     def E_step(self, X, theta, param):
#         '''
#         E步：更新隐变量概率分布q(Z)。
#         '''
#         q = np.zeros((param['k'], len(X)))
#         for i in range(param['k']):
#             q[i, :] = self.GMM_component(X, theta, param, i)
#         q /= q.sum(axis=0)
#         return q

#     def M_step(self, X, q, theta, param):
#         '''
#         M步：使用q(Z)更新GMM参数。
#         '''
#         pi_temp = q.sum(axis=1); pi_temp /= len(X) # 计算pi
#         mu_temp = q.dot(X); mu_temp /= q.sum(axis=1)[:, None] # 计算mu
#         sigma_temp = np.zeros((param['k'], param['dim'], param['dim']))
#         for i in range(param['k']):
#             ys = X - mu_temp[i, :]
#             sigma_temp[i] = np.sum(q[i, :, None, None]*np.matmul(ys[..., None], ys[:, None, :]), axis=0)
#         sigma_temp /= np.sum(q, axis=1)[:, None, None] # 计算sigma
#         theta['pi'] = pi_temp; theta['mu'] = mu_temp; theta['sigma'] = sigma_temp
#         return theta

#     def likelihood(self, X, theta, param):
#         '''
#         计算GMM的对数似然。
#         '''
#         ll = 0
#         for i in range(param['k']):
#             ll += self.GMM_component(X, theta, param, i)
#         ll = np.log(ll).sum()
#         return ll

#     def EM_GMM(self, X, param, eps=1e-5, max_iter=100):
#         '''
#         高斯混合模型的EM算法求解
#             theta: GMM模型参数; param: 其它系数; eps: 计算精度; max_iter: 最大迭代次数
#             返回对数似然和参数theta，theta是包含pi、mu、sigma的Python字典
#         '''
#         theta = {}
#         theta['pi'] = np.ones(param['k'])/param['k']                 # 均匀初始化
#         theta['mu'] = np.random.random((param['k'],param['dim']))    # 随机初始化
#         theta['sigma'] = np.array([np.eye(param['dim'])]*param['k']) # 初始化为单位正定矩阵
#         for i in range(max_iter):
#             ll_old = 0
#             # E-step
#             q = self.E_step(X, theta, param)
#             # M-step
#             theta = self.M_step(X, q, theta, param)
#             ll_new = self.likelihood(X, theta, param)
#             if np.abs(ll_new - ll_old) < eps:
#                 break;
#             else:
#                 ll_old = ll_new
#         return theta

    """
    EM_GMM(self, X, param, eps=1e-5, max_iter=100)对相同标签的数据进行混合高斯聚类
    
    Parameters:
      X - 输入的训练数据
      param - 训练数据特征参数
      eps - EM算法收敛阈值
    Returns:
      混合高斯模型参数，包括pi、mu、sigma
    """
    def EM_GMM(self, X, param, eps=1e-5, max_iter=100):
        theta = {}
        theta['pi'] = np.ones(param['k'])/param['k']                 # 均匀初始化
        theta['mu'] = np.random.random((param['k'],param['dim']))    # 随机初始化
        theta['sigma'] = np.array([np.eye(param['dim'])]*param['k']) # 初始化为单位正定矩阵
        
        Q_mat = np.zeros((len(X), param['k']))  # 概率矩阵
        for i in range(max_iter):# 迭代次数
            for k in range(param['k']):
                theta['sigma'][k, ...] += self.regularization
                #### E-step，求Q矩阵 ####
                Q_mat[:, k] = theta['pi'][k]*multivariate_normal(theta['mu'][k], theta['sigma'][k, ...], allow_singular = True).pdf(X)
            totol_N = Q_mat.sum(axis=1)  # 计算各样本出现的总频率
            totol_N[totol_N == 0] = param['k']# 如果某一样本在各类中的出现频率和为0，则使用K来代替，相当于分配等概率
            Q_mat /= totol_N.reshape(-1, 1)
            
            #### M-step，更新参数 ####
            for k in range(param['k']):
                Nk = np.sum(Q_mat[:, k], axis=0)  # 类出现的频率
                theta['pi'][k] = Nk / len(X)
                theta['mu'][k] = (1 / Nk) * np.sum(X *Q_mat[:, k].reshape(-1, 1), axis=0)  # 该类的新均值
                theta['sigma'][k] = (1 / Nk) * np.dot((Q_mat[:, k].reshape(-1, 1)* (X - theta['mu'][k])).T,(X - theta['mu'][k]))
            return theta    
        
    def train(self, X, y):
        param = {}
        param['k'] = self.k; 
        param['dim'] = X.shape[1]
        self.regularization = np.dot(np.eye(param['dim']),0.001)#正则化项
  
        labels = list(set(y))#标签的列表
        data = {label:[] for label in labels}#{0.0: [], 1.0: []}
        for f, label in zip(X, y):
            data[label].append(f)#print(data)#形成一个字典，根据标签将训练样本进行分类
       
        self.model = {label:{} for label in range(len(labels))}        
        for i in range(len(labels)):
            self.model[i] = self.EM_GMM(data[i],param,eps=1e-5, max_iter=self.max_iter)
            
        return self.model

    # 计算概率
    def calculate_probabilities(self, input_data):
        probabilities = {}
        dim = np.size(input_data)
        
        for label, value in self.model.items():#value是一个字典，表示特定标签的模型参数
            pp = 0.0
            for i in range(self.k):
                mu,sigma = value['mu'][i],value['sigma'][i]
                pp += multivariate_normal(mu, sigma, allow_singular = True).pdf(input_data)*value['pi'][i]
            probabilities[label] = pp
            
        return probabilities
    
    # 类别
    def predict(self, X_test):
        label = list(range(X_test.shape[0]))
        for i in range(X_test.shape[0]):#每个样本迭代一次
            label[i] = sorted(self.calculate_probabilities(X_test[i,:]).items(), key=lambda x: x[-1])[-1][0]
        
        return label
    
    
iris = datasets.load_iris()
X=iris.data
y=iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=1)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
model = GMM(n_components=5, max_iter = 10)
model_0 = model.train(X_train, y_train)
y_pred = model.predict(X_test)
print("IRIS:Number of mislabeled points out of a total %d points : %d, Acc: %f%%"
      % (X_test.shape[0], (y_test != y_pred).sum(),100*(y_test == y_pred).sum()/X_test.shape[0]))

(75, 4) (75, 4) (75,) (75,)
IRIS:Number of mislabeled points out of a total 75 points : 0, Acc: 100.000000%


### 利用sk-learn mixture gaussion进行拟合

In [25]:
class GMM:
    def __init__(self,n_components = 5, max_iter = 100):
        self.k = 3
        self.max_iter = max_iter
        
    def train(self, X, y):
        
        labels = list(set(y))#标签的列表
        data = {label:[] for label in labels}#{0.0: [], 1.0: []}
        for f, label in zip(X, y):
            data[label].append(f)#print(data)#形成一个字典，根据标签将训练样本进行分类
       
        self.model = {label:{} for label in range(len(labels))}
        
        for i in range(len(labels)):
            dpgmm = GaussianMixture(n_components=self.k).fit(data[i])
            self.model[i]['pi'] = dpgmm.weights_
            self.model[i]['mu'] = dpgmm.means_
            self.model[i]['sigma'] = dpgmm.covariances_

        return self.model

    # 计算概率
    def calculate_probabilities(self, input_data):
        probabilities = {}
        dim = np.size(input_data)
        
        for label, value in self.model.items():#value是一个字典，表示特定标签的模型参数
            pp = 0.0
            for i in range(self.k):
                mu,sigma = value['mu'][i],value['sigma'][i]
                pp += multivariate_normal(mu,sigma).pdf(input_data)*value['pi'][i]
            probabilities[label] = pp
            
        return probabilities
    
    # 类别
    def predict(self, X_test):
        label = list(range(X_test.shape[0]))
        for i in range(X_test.shape[0]):#每个样本迭代一次
            label[i] = sorted(self.calculate_probabilities(X_test[i,:]).items(), key=lambda x: x[-1])[-1][0]
        
        return label
    

## 混合高斯分类鸢尾花数据ISRS

In [22]:
iris = datasets.load_iris()
X=iris.data
y=iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=1)
# print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

model = GMM(n_components = 5, max_iter = 100)
model.train(X_train, y_train)
y_pred = model.predict(X_test)
print("IRIS:Number of mislabeled points out of a total %d points : %d, Acc: %f%%"
      % (X_test.shape[0], (y_test != y_pred).sum(),100*(y_test == y_pred).sum()/X_test.shape[0]))

IRIS:Number of mislabeled points out of a total 75 points : 0, Acc: 100.000000%


## 混合高斯分类MNIST数据

In [26]:
from sklearn.decomposition import PCA#主成分分析
from sklearn.datasets import fetch_mldata
 
mnist = fetch_mldata('MNIST original',data_home="E:\scikit_learn_data")
X, y = mnist["data"], mnist["target"]

X_reduced = PCA(n_components=50).fit_transform(X)#特征提取
X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X_reduced, y, test_size=0.1, random_state=1)#random_state=1使数据集顺序打乱
# X_train_d = (X_train_d-np.min(X_train_d))/(np.max(X_train_d)-np.min(X_train_d))#数值归一化
# X_test_d = (X_test_d-np.min(X_test_d))/(np.max(X_test_d)-np.min(X_test_d))

model_d = GMM(n_components = 5, max_iter = 100)
# print(model_d.train(X_train_d, y_train_d))
model_d.train(X_train_d, y_train_d)
y_pred_d = model_d.predict(X_test_d)

print("MNIST:Number of mislabeled points out of a total %d points : %d, Acc: %f%%"
      %(X_test_d.shape[0], (y_test_d != y_pred_d).sum(),100*(y_test_d == y_pred_d).sum()/X_test_d.shape[0]))

C:\Users\12569\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:85: DeprecationWarning: Function fetch_mldata is deprecated; fetch_mldata was deprecated in version 0.20 and will be removed in version 0.22. Please use fetch_openml.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\12569\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:85: DeprecationWarning: Function mldata_filename is deprecated; mldata_filename was deprecated in version 0.20 and will be removed in version 0.22. Please use fetch_openml.
  warnings.warn(msg, category=DeprecationWarning)


{0: {'pi': array([0.38542469, 0.27546179, 0.33911352]), 'mu': array([[ 1.09756934e+03, -2.42785576e+02, -2.68580703e+02,
         1.20881542e+02, -7.49221564e+02,  4.19960203e+02,
         1.86762357e+02, -3.43633295e+02,  2.19792833e+01,
         1.37929085e+02, -7.00066622e+01,  8.20211315e+01,
        -6.36765878e+01, -1.59063510e+02, -5.84328991e+01,
         3.88015487e+01,  1.39584488e+02, -3.44705956e+01,
        -4.74845549e+01, -7.01420532e+01, -5.56512015e+01,
        -9.13699351e+00, -1.03100278e+02, -1.45365276e+01,
         5.39773276e+01, -2.14957846e+01, -1.63815821e+01,
        -4.62784095e+01,  7.78056062e+01,  1.05417475e+01,
         1.80830152e+01, -1.04340300e+01,  2.46686249e+01,
         1.54206945e+01, -5.16094200e-01, -2.97241284e+00,
         2.30277250e+01,  1.03596616e+01, -2.96119015e+00,
         3.17510548e+01,  5.23743880e+01, -2.53054392e+01,
         1.75803496e+01, -2.76357722e+01, -2.62764423e+01,
         5.17739788e+00,  4.42833989e+00, -2.74357331

MNIST:Number of mislabeled points out of a total 7000 points : 249, Acc: 96.442857%


## 混合高斯分类CIFAR10数据

In [27]:
import pickle

def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

cifar_batch = {}
cifar_batch[0] = unpickle("E:\scikit_learn_data\cifar-10-batches-py\data_batch_5")
cifar_batch[1] = unpickle("E:\scikit_learn_data\cifar-10-batches-py\data_batch_1")
cifar_batch[2] = unpickle("E:\scikit_learn_data\cifar-10-batches-py\data_batch_2")
cifar_batch[3] = unpickle("E:\scikit_learn_data\cifar-10-batches-py\data_batch_3")
cifar_batch[4] = unpickle("E:\scikit_learn_data\cifar-10-batches-py\data_batch_4")
cifar_test = unpickle("E:\scikit_learn_data\cifar-10-batches-py\\test_batch")
cifar_batch_meta = unpickle("E:\scikit_learn_data\cifar-10-batches-py\\batches.meta")

model_c = GMM()
for i in range(5):
    X_c,y_c = cifar_batch[i][b'data'],cifar_batch[i][b'labels']
    X_c = X_c/255#对图片数据进行归一化
    y_c = np.array(y_c)#将y_c转化为numpy数组
    X_reduced_c = PCA(n_components=30).fit_transform(X_c)#特征提取
#     X_reduced_c = (X_reduced_c-np.min(X_reduced_c))/(np.max(X_reduced_c)-np.min(X_reduced_c))
    model_c.train(X_reduced_c, y_c)

X_test,y_test = cifar_test[b'data'],cifar_test[b'labels']
# X_test,y_test = cifar_batch[4][b'data'],cifar_batch[4][b'labels']
X_test = X_test/255
y_test_c = np.array(y_test)
X_test_c = PCA(n_components=30).fit_transform(X_test)#特征提取
# X_test_c = (X_test_c-np.min(X_test_c))/(np.max(X_test_c)-np.min(X_test_c))#数据归一化
y_pred_c = model_c.predict(X_test_c)

print("CIFAR:Number of mislabeled points out of a total %d points : %d, Acc: %f%%"
      %(X_test_c.shape[0], (y_test_c != y_pred_c).sum(),100*(y_test_c == y_pred_c).sum()/X_test_c.shape[0]))

CIFAR:Number of mislabeled points out of a total 10000 points : 8635, Acc: 13.650000%
